In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test

def accuracy_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

def Logistic(X_train, y_train, X_test):       
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    accuracy = accuracy_prediction(classifier, X_test, y_test)
    return accuracy   

def svm_linear(X_train, y_train, X_test):                
    classifier = SVC(kernel='linear')
    classifier.fit(X_train, y_train)
    accuracy = accuracy_prediction(classifier, X_test, y_test)
    return accuracy
    
def svm_NL(X_train, y_train, X_test):                
    classifier = SVC(kernel='rbf')
    classifier.fit(X_train, y_train)
    accuracy = accuracy_prediction(classifier, X_test, y_test)
    return accuracy  

def Decision(X_train, y_train, X_test):        
    classifier = DecisionTreeClassifier(random_state=0)
    classifier.fit(X_train, y_train)
    accuracy = accuracy_prediction(classifier, X_test, y_test)
    return accuracy  

def random(X_train, y_train, X_test):       
    classifier = RandomForestClassifier(n_estimators=10, random_state=0)
    classifier.fit(X_train, y_train)
    accuracy = accuracy_prediction(classifier, X_test, y_test)
    return accuracy 

def selectk_classification(acclog, accsvml, accsvmnl, accdes, accrf): 
    dataframe = pd.DataFrame(index=['SelectFromModel'], columns=['Logistic', 'SVMl', 'SVMnl', 'Decision', 'Random'])

    for number, idx in enumerate(dataframe.index):
        dataframe['Logistic'][idx] = acclog[number]       
        dataframe['SVMl'][idx] = accsvml[number]
        dataframe['SVMnl'][idx] = accsvmnl[number]
        dataframe['Decision'][idx] = accdes[number]
        dataframe['Random'][idx] = accrf[number]

    return dataframe

dataset1 = pd.read_csv("prep.csv", index_col=None)
df2 = pd.get_dummies(dataset1, drop_first=True)

indep_X = df2.drop("classification_yes", axis=1)
dep_Y = df2['classification_yes']

# Use SelectFromModel for feature importance selection
select_model = LogisticRegression()  # You can use any classification model for feature importance
sfm = SelectFromModel(select_model, threshold='mean')
sfm.fit(indep_X, dep_Y)
kbest = sfm.transform(indep_X)

acclog = []
accsvml = []
accsvmnl = []
accdes = []
accrf = []
X_train, X_test, y_train, y_test = split_scalar(kbest, dep_Y)

for i in kbest:   
    accuracy_log = Logistic(X_train, y_train, X_test)
    acclog.append(accuracy_log)
    
    accuracy_sl = svm_linear(X_train, y_train, X_test)    
    accsvml.append(accuracy_sl)
    
    accuracy_NL = svm_NL(X_train, y_train, X_test)
    accsvmnl.append(accuracy_NL)
    
    accuracy_d = Decision(X_train, y_train, X_test)
    accdes.append(accuracy_d)
    
    accuracy_r = random(X_train, y_train, X_test)
    accrf.append(accuracy_r)

result = selectk_classification(acclog, accsvml, accsvmnl, accdes, accrf)
print(result)


D:\ANACONDA\ANA\envs\aiworks\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


                Logistic  SVMl SVMnl Decision Random
SelectFromModel     0.98  0.97  0.98     0.91   0.98
